In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.9 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelWithLMHead
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch

In [ ]:
from torch.utils.data import Dataset

class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        f= open(path, "r")
        self.data = f.readlines()

        self.X = []
        for i in self.data:
                self.X.append(i)

        for idx, i in enumerate(self.X):
            try:
                self.X[idx] = "<startofstring> "+i+" <bot>: "+self.X[idx+1]+" <endofstring>"
            except:
                break

        self.X = self.X

        print(self.X[0])

        self.X_encoded = tokenizer(self.X,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [ ]:
def train(chatData, model, optim):

    epochs = 3

    for i in tqdm.tqdm(range(epochs)):
        for X, a in tqdm.tqdm(chatData,miniters = 100):
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        ##  https://drive.google.com/drive/folders/1vkrCAwNGt658vOifr8HbiQKnr8-7Jv3L make a shortcut from this link and use the path here
        model.save_pretrained("/content/drive/MyDrive/tandem_model", from_pt=True)
        print(infer("hallo Wie geht's dir?"))

def infer(inp):
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])
    return output

In [ ]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                                "bos_token": "<startofstring> ",
                                "eos_token": " <endofstring>"})
tokenizer.add_tokens([" <bot>: "])
##  https://drive.google.com/drive/folders/1vkrCAwNGt658vOifr8HbiQKnr8-7Jv3L make a shortcut from this link and use the path here
model = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/tandem_model")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50269. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


In [ ]:
# notice that I am using the data from the original file
chatData = ChatData("/content/drive/MyDrive/tandem_model/data/train.txt", tokenizer)
chatData =  DataLoader(chatData, batch_size=64)
model.train()

optim = Adam(model.parameters(), lr=1e-3)

print("training .... ")
train(chatData, model, optim)

print("infer from model : ")

<startofstring> an diesem grundsatze muss auch dann festgehalten werden , wenn der erwiderte schlag stärker gewesen wäre , oder selbst eine verwundung nach sich gezogen hätte . ”
 <bot>: die verletzung bildet weder eine neue beleidigung noch eine vermehrung derselben . ”
 <endofstring>
training .... 


100%|██████████| 3796/3796 [43:36<00:00,  1.45it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
 33%|███▎      | 1/3 [43:44<1:27:29, 2624.53s/it]

<startofstring> hallo Wie geht's dir? <bot>: 
 <bot>: 
 <endofstring><pad><pad><pad><pad><pad><pad><pad>



100%|██████████| 3796/3796 [43:37<00:00,  1.45it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
 67%|██████▋   | 2/3 [1:27:24<43:41, 2621.57s/it]

<startofstring> hallo Wie geht's dir? <bot>: 
 <bot>: ~how presa! ~
 <endofstring><pad>



100%|██████████| 3796/3796 [43:37<00:00,  1.45it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 3/3 [2:11:04<00:00, 2621.53s/it]

<startofstring> hallo Wie geht's dir? <bot>: 
 <bot>: 
 <endofstring><pad><pad><pad><pad><pad><pad><pad>
infer from model : 


In [ ]:
print(infer('Erzähle mir einen Witz.'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> Erzähle mir einen Witz. <bot>:  mich deines rüdenwäldes zu einem narren, daß


## Evaluation

In [7]:
# Wheeeee~!
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModelWithLMHead
import json
from random import randint
import math
import csv
import pandas as pd
import re
from tqdm import tqdm
import torch
import numpy as np
import cefr
pattern = re.compile('[\W_]+')

def calculate_perplexity(predictions, actual_tokens):
    perplexities = []
    for beam in range(10):
        log_probability = 0.0
        num_tokens = len(predictions)

        for i in range(num_tokens):
            token_logits = predictions[i]
            actual_token = actual_tokens[i]
            token_logit = token_logits[beam][actual_token]
            # Calculate the log likelihood of the actual token
            if torch.isinf(token_logit) != True and token_logit.item() != -1000000000.0:
                log_probability += token_logit.item()
                #print("token logit:", token_logit.item())
            # probability = math.exp(token_logit.item())/(1 + math.exp(token_logit.item()))
            # print("probability:", probability)
            # if probability != 0:
            #     log_probability += math.log(probability)
        # Calculate perplexity as the exponential of the negative log-likelihood
        # perplexity = math.exp(-log_probability / num_tokens)
        # perplexities.append(perplexity)
        # perplexities.append(-log_probability / num_tokens)
        avg_log_probability = log_probability / num_tokens
        perplexities.append(-avg_log_probability)

    return np.mean(perplexities)

def generate_constraint_list():
    with open('constraint_lists.json', 'r', encoding="utf-8") as f:
        constraint_lists = json.load(f)
        # Picks a random unit (1-12) list from A1 word list
    unit = randint(1, len(constraint_lists["A1 Full List"]))
    return constraint_lists["A1 Full List"][f"Unit {unit}"]

def generate_constraint_sample(constraint_list):
    list_length = len(constraint_list)
    index = randint(0, list_length-30)
    return constraint_list[index:index+30]

def main():

    tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2")
    tokenizer.add_special_tokens({"pad_token": "<pad>",
                                    "bos_token": "<startofstring> ",
                                    "eos_token": " <endofstring>"})
    tokenizer.add_tokens([" <bot>: "])
    ##  https://drive.google.com/drive/folders/1vkrCAwNGt658vOifr8HbiQKnr8-7Jv3L make a shortcut from this link and use the path here
    model = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/tandem_model")
    model.resize_token_embeddings(len(tokenizer))


    # from transformers import AutoTokenizer, AutoModelWithLMHead
    # tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2")
    # model = AutoModelWithLMHead.from_pretrained("dbmdz/german-gpt2")

    # Takes a 30-word slice as constraint list for generation
    constraint_list = generate_constraint_list()
    force_words_ids = [
        tokenizer(generate_constraint_sample(constraint_list), add_special_tokens=False).input_ids,
    ]

    # Llama prompt:
    #prompt = "[INST] <<SYS>> You are a friendly German language partner. Your role is to chat with language learners to help them improve their language skills. Your answers should be brief, friendly and conversational. Please answer in B1-level German. <</SYS>>"
    # Non-llama prompt:
    # prompt = ["You are a friendly German language partner. Your role is to chat with language learners to help them improve their language skills. Your answers should be brief, friendly and conversational. Please answer in A1-level German."]
    # Most basic prompt:
    prompt = "Hallo"

    with open('Test set.csv', 'r', encoding="utf-8") as f:
        test_data = list(csv.reader(f, delimiter=","))

    responses = []
    perplexities = []
    grades = []

    run_size = 49
    for question in tqdm(test_data[:run_size]):
        # Llama input
        input = f'{prompt}{question}[/INST]'
        # Non-llama input
        # input = f'{prompt}{question}'
        tokenized_prompt = tokenizer.encode(input, return_tensors="pt")
        # Generate a response
        print("Generating response...")
        while True:
            try:
                outputs = model.generate(
                    tokenized_prompt,
                    force_words_ids=force_words_ids,
                    num_beams=10,
                    num_return_sequences=1,
                    no_repeat_ngram_size=1,
                    remove_invalid_values=True,
                    max_new_tokens=100,
                    return_dict_in_generate=True,
                    output_scores=True
                )
                break
            except ValueError:
                print("ValueError. Restarting current loop with new constraint list...")
                force_words_ids = [
                tokenizer(generate_constraint_sample(constraint_list), add_special_tokens=False).input_ids,
                ]
                pass

        # Extract and print the chatbot's response
        logits = outputs.scores
        bot_response = tokenizer.decode(outputs.sequences[0])[-len(logits):]
        responses.append(bot_response)
        print("Chatbot:", bot_response)
        # calculate perplexity
        # Calculate perplexity for the model's predictions
        # print("logits length:",len(logits))
        # print("output sequence:", outputs.sequences[0][-len(logits):])
        # print("decoded output:", tokenizer.decode(outputs.sequences[0][-len(logits):]))
        # print("sequence length:",len(outputs.sequences[0][-len(logits):])) #[len(tokenized_prompt):]
        perplexity = calculate_perplexity(logits, outputs.sequences[0][-len(logits):])
        perplexities.append(perplexity)
        response = bot_response.split()
        response = [pattern.sub('', word) for word in response]
        grade = cefr.CEFR_level_guesser(response)
        grades.append(grade)
        print("Perplexity per word:", perplexity)

    results_dict = {"input":test_data[:run_size], "response":responses, "perplexity":perplexities, "grade":grades}
    output_data = pd.DataFrame.from_dict(results_dict)
    output_data.to_csv("output_data.csv", encoding="utf-8")

if __name__ == "__main__":
    main()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50269. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
  0%|          | 0/49 [00:00<?, ?it/s]The attention mask a

Generating response...
ValueError. Restarting current loop with new constraint list...
ValueError. Restarting current loop with new constraint list...
ValueError. Restarting current loop with new constraint list...


  2%|▏         | 1/49 [00:16<13:17, 16.62s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: nsermütter ’ s? gliedern das bekennt ebenso schlecht wie harm-spren alle hier amtriecht schöne _hart
Perplexity per word: 68.14634871959687
Generating response...


  4%|▍         | 2/49 [00:32<12:49, 16.37s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Chatbot: en untreue kunststücke geben müssen bis morgen oder er verdiente. was beharrieren mit kostühart pfeß
Perplexity per word: 59.32763170647621
Generating response...


 98%|█████████▊| 48/49 [13:19<00:16, 16.93s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot:  oberen thalbt besser für heute : wer keines menschenherzens weiter nichts — gewiß nicht durchgemein
Perplexity per word: 57.021401737928386
Generating response...


100%|██████████| 49/49 [13:36<00:00, 16.66s/it]

Chatbot: zen hofbuch unserer affentödesoëtheilung der eine radetzzelereien wohlthäufen sich wieder : ›indhart
Perplexity per word: 56.698452854752546
